In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
!pip install plotly

In [33]:
import pandas as pd
import numpy as np
import json
import folium
import plotly.offline as pyo
import plotly.graph_objs as go

In [ ]:
# 서울시 구별 인구
ppl = pd.read_csv("/content/drive/My Drive/project/seoul_contest/서울시 주민등록인구 (연령별동별) 통계.txt",header=None , sep='\t')
ppl.loc[0]
ppl.columns
ppl = ppl.loc[:,[1,  2,  3,  4,  ]]
ppl.columns = ppl.loc[0]
ppl = ppl.loc[1:].set_index('자치구')
pop = ppl[ppl['구분']=='계']
pop = pop[pop['동']=='소계']
pop_by_gu=pd.DataFrame(pop['계'])
pop_by_gu.to_pickle('population.pkl')

In [277]:
# 서울시 동별 인구
ppl = pd.read_csv("/content/drive/My Drive/project/seoul_contest/서울시 주민등록인구 (연령별동별) 통계.txt",header=None , sep='\t')
ppl.loc[0]
ppl.columns
ppl = ppl.loc[:,[1,  2,  3,  4,  ]]
ppl.columns = ppl.loc[0]
ppl = ppl.loc[1:].set_index('동')
pop = ppl[ppl['구분']=='계']
pop_by_dong = pop.loc[:,['자치구', '계']]
pop_by_dong.to_pickle('population_by_dong.pkl')

서울시 5대 범죄율 (구별) 2019년

In [176]:
# crime = pd.read_csv("/content/drive/My Drive/project/seoul_contest/report서울시 5대 범죄 발생현황 통계(구별).txt",names=['5대범죄',	'5대범죄현황',	'합계'	,'종로구',	'중구',	'용산구'	,'성동구',	'광진구',	'동대문구',	'중랑구',	'성북구',	'강북구',	'도봉구',	'노원구',	'은평구',	'서대문구',	'마포구',	'양천구',	'강서구',	'구로구',	'금천구',	'영등포구',	'동작구'	,'관악구',	'서초구',	'강남구',	'송파구',	'강동구'])
crime = pd.read_csv("/content/drive/My Drive/project/seoul_contest/report서울시 5대 범죄 발생현황 통계(구별).txt",header=None , sep='\t', thousands = ',')
crime = crime[1:]
c = crime.T
crime_df = c[2:]
col=['5대범죄','합계(발생)'  ,'합계(검거)' , '살인(발생)' , '살인(검거)',  '강도(발생)' , '강도(검거)' , '강간강제추행(발생)' , '강간강제추행(검거)' , '절도(발생)' , '절도(검거)' , '폭력(발생)', '폭력(검거)']
crime_df.columns = col 

,5대범죄,합계(발생),합계(검거),살인(발생),살인(검거),강도(발생),강도(검거),강간강제추행(발생),강간강제추행(검거),절도(발생),절도(검거),폭력(발생),폭력(검거)
2,합계,"103,668","74,805",136,128,136,133,"6,469","6,007","42,204","21,284","54,723","47,253"
3,종로구,"3,846","4,117",4,5,7,8,238,"1,143","1,515","1,069","2,082","1,892"
4,중구,"4,327","2,804",2,1,6,5,195,115,"2,202","1,050","1,922","1,633"
5,용산구,"3,313","2,611",3,3,3,4,272,237,999,544,"2,036","1,823"
6,성동구,"2,512","1,838",6,5,9,10,133,96,970,511,"1,394","1,216"
7,광진구,"4,011","2,816",4,5,6,5,273,213,"1,875","1,008","1,853","1,585"
8,동대문구,"3,692","2,818",6,6,10,9,168,143,"1,531",827,"1,977","1,833"
9,중랑구,"4,268","3,138",10,10,4,5,138,108,"1,666",908,"2,450","2,107"
10,성북구,"2,877","2,323",3,3,3,3,176,130,"1,082",731,"1,613","1,456"
11,강북구,"3,838","3,023",6,6,3,2,176,145,"1,229",664,"2,424","2,206"


In [253]:
crime_df['합계(검거)'] = crime_df['합계(검거)'].str.replace(',', '').astype('int64')
crime_df['합계(발생)']= crime_df['합계(발생)'].str.replace(',', '').astype('int64')
crime_df['검거율'] = (crime_df['합계(검거)']/crime_df['합계(발생)'])*100
crime_df = crime_df.set_index('5대범죄')
# crime_df.to_pickle('crime_top5.pkl')

In [252]:
df = pd.merge(crime_df,pop_by_gu,left_index=True, right_index=True)
df['계'] = df['계'].str.replace(',', '').astype('int64')
df["인구대비 범죄 발생률"] = df['합계(발생)']/df['계']
df
# df.to_pickle('crime_per_pop.pkl')

,합계(발생),합계(검거),살인(발생),살인(검거),강도(발생),강도(검거),강간강제추행(발생),강간강제추행(검거),절도(발생),절도(검거),폭력(발생),폭력(검거),검거율,계,인구대비 범죄 발생률
종로구,3846,4117,4,5,7,8,238,"1,143","1,515","1,069","2,082","1,892",107.046282,160520,0.023960
중구,4327,2804,2,1,6,5,195,115,"2,202","1,050","1,922","1,633",64.802404,136030,0.031809
용산구,3313,2611,3,3,3,4,272,237,999,544,"2,036","1,823",78.810746,245362,0.013502
성동구,2512,1838,6,5,9,10,133,96,970,511,"1,394","1,216",73.168790,304851,0.008240
광진구,4011,2816,4,5,6,5,273,213,"1,875","1,008","1,853","1,585",70.206931,363925,0.011022
동대문구,3692,2818,6,6,10,9,168,143,"1,531",827,"1,977","1,833",76.327194,360380,0.010245
중랑구,4268,3138,10,10,4,5,138,108,"1,666",908,"2,450","2,107",73.523899,399415,0.010686
성북구,2877,2323,3,3,3,3,176,130,"1,082",731,"1,613","1,456",80.743830,452290,0.006361
강북구,3838,3023,6,6,3,2,176,145,"1,229",664,"2,424","2,206",78.764982,315395,0.012169
도봉구,2110,1497,1,1,5,5,90,83,790,394,"1,224","1,014",70.947867,331744,0.006360


In [200]:
trace1 = go.Bar(x=crime_df['5대범죄'][1:], y=crime_df['합계(발생)'][1:])
data = [trace1]
layout = go.Layout(title='구별 범죄')
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [247]:
trace2 = go.Bar(x=crime_df['5대범죄'][1:], y=crime_df['검거율'][1:])
data = [trace]
layout = go.Layout(title='구별 범죄 검거율')
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [250]:
trace3 = go.Bar(x=df.index, y=df["인구대비 범죄 발생률"])
data = [trace3]
layout = go.Layout(title='구별 인구대비 범죄 발생률')
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

서울시 주차장 통계

In [391]:
#  공영주차장 합계= 노상주차장 + 노외주차장 (시영,구영), 민영주차장 합계= 노외주차장 (민영) + 건축물부설
parking = pd.read_csv("/content/drive/My Drive/project/seoul_contest/서울시 주차장 (동별) 통계.txt" ,header=None, sep='\t')
parking = parking.loc[:,[1,2,3,4,5,6,7,8]]#.set_index(parking.loc[:,2])
col1 = ['자치구','동','합계(개소)','합계(면수)','합계(공영개수)','합계(공영면수)','합계(민영개수)','합계(민영면수)']
parking.columns = col1
parking = parking.set_index('동')[3:]
# parking.to_pickle('parkinglot_sum.pkl')

In [394]:
parking_dong = parking.drop("소계", axis=0)
pop_by_dong = pop_by_dong.drop("소계", axis=0)

In [403]:
# 구 단위 주차면적
parking_gu= parking.loc["소계"].set_index('자치구')
lot_gu = pd.merge(pop_by_gu,parking_gu,left_index = True,right_index=True)
lot_gu['합계(면수)'] = lot_gu['합계(면수)'].str.replace(',', '').astype('int64')
lot_gu['계']= lot_gu['계'].str.replace(',', '').astype('int64')
lot_gu['1인당 주차면적'] = lot_gu['합계(면수)']/lot_gu['계']
lot_gu

,계,합계(개소),합계(면수),합계(공영개수),합계(공영면수),합계(민영개수),합계(민영면수),1인당 주차면적
자치구,,,,,,,,
종로구,160520,"7,103",95350,194,"6,142","6,909","89,208",0.594007
중구,136030,"4,228",107399,178,"5,408","4,050","101,991",0.789524
용산구,245362,"7,390",113591,424,"6,532","6,966","107,059",0.462953
성동구,304851,"6,649",133724,287,"8,174","6,362","125,550",0.438654
광진구,363925,"17,495",138741,"1,350","5,619","16,145","133,122",0.381235
동대문구,360380,"10,455",138200,685,"6,026","9,770","132,174",0.383484
중랑구,399415,"12,381",133615,562,"11,119","11,819","122,496",0.334527
성북구,452290,"13,014",156915,219,"7,283","12,795","149,632",0.346934
강북구,315395,"10,969",91187,49,"5,108","10,920","86,079",0.289120


In [407]:
# 동 단위 주차면적
lot = pd.merge(pop_by_dong,parking_dong,left_index = True,right_index=True)
lot['합계(면수)'] = lot['합계(면수)'].str.replace(',', '').astype('int64')
lot['계']= lot['계'].str.replace(',', '').astype('int64')
lot['1인당 주차면적'] = lot['합계(면수)']/lot['계']
# lot.to_pickle('1인당 주차면적.pkl')

In [382]:
lot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 가락1동 to 흑석동
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   자치구_x     426 non-null    object
 1   계         426 non-null    object
 2   자치구_y     426 non-null    object
 3   합계(개소)    426 non-null    object
 4   합계(면수)    426 non-null    object
 5   합계(공영개수)  426 non-null    object
 6   합계(공영면수)  426 non-null    object
 7   합계(민영개수)  426 non-null    object
 8   합계(민영면수)  426 non-null    object
dtypes: object(9)
memory usage: 33.3+ KB


In [409]:
trace5 = go.Bar(x=lot_gu.index, y=lot_gu["1인당 주차면적"])
data = [trace5]
layout = go.Layout(title='구별 1인당 주차면적(m^2)')
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [408]:
trace4 = go.Bar(x=lot.index, y=lot["1인당 주차면적"])
data = [trace4]
layout = go.Layout(title='동별 1인당 주차면적(m^2)')
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [410]:
trace4 = go.Bar(x=lot['자치구_x'], y=lot["1인당 주차면적"])
data = [trace4]
layout = go.Layout(title='구별 1인당 주차면적(m^2)')
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)


In [ ]:
center = (37.5642135, 127.0016985)
zoom = 13

m = folium.Map(location=center, zoom_start=11)
folium.Choropleth(
    geo_data = gu_boundary,
    data = df,
    columns = ('자치구', 'count_per_정원'),
    key_on = 'feature.properties.SIG_KOR_NM',
    fill_color = 'BuPu',
    legend_name = 'None'
).add_to(m)
for index, row in df.iterrows():
  tip = f'{row["SIG_ENG_NM"]} count_per_num:{row["count_per_정원"]}'
  folium.Marker([row['위도'], row['경도']], tooltip=tip).add_to(m) 
m